In [1]:
import pandas as pd
import osmnx as ox
import numpy as np
import json
import geopandas as gpd
from shapely.geometry import Point
from geopandas.testing import assert_geodataframe_equal
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../') 
from src.graph.create_osmnx_graph import OSMnxGraph 


In [2]:
cities_pl = ['Warszawa', 'Kraków', 'Wrocław', 'Poznań', 'Szczecin']

all_nodes_pl = pd.DataFrame()
all_edges_pl = pd.DataFrame()

for city in cities_pl:
    G =ox.graph.graph_from_place(f"{city}, Poland", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    all_nodes_pl = pd.concat([all_nodes_pl, gdf_nodes])
    all_edges_pl = pd.concat([all_edges_pl, gdf_edges])


In [3]:
cities_usa = ['Seattle', 'New York City']

all_nodes_usa = pd.DataFrame()
all_edges_usa = pd.DataFrame()

for city in cities_usa:
    G =ox.graph.graph_from_place(f"{city}, United States", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    all_nodes_usa = pd.concat([all_nodes_usa, gdf_nodes])
    all_edges_usa = pd.concat([all_edges_usa, gdf_edges])

In [4]:
all_attributes = {}
for col in ['highway', 'access', 'junction', 'bridge', 'tunnel']:
    edges_usa = all_edges_usa[col].dropna().unique()
    edges_pl = all_edges_pl[col].dropna().unique()
    all_edges = set(np.concatenate((edges_pl, edges_usa)))
    all_attributes[f'edges_{col}'] = list(all_edges)

     
nodes_highway_usa = all_nodes_usa['highway'].dropna().unique()
nodes_highway_pl = all_nodes_pl['highway'].dropna().unique()   
all_nodes_highway = set(np.concatenate((nodes_highway_pl, nodes_highway_usa)))
all_attributes['nodes_highway'] = list(all_nodes_highway)

with open('../data/osmnx_attributes.json', 'w') as f:
    json.dump(all_attributes, f)
    

In [2]:
with open("../data/osmnx_attributes.json") as f:
    all_attributes = json.load(f)
    
num_edges_attrs = 0
for key in all_attributes.keys():
    if 'edges' in key:
        num_edges_attrs += len(all_attributes[key])
        
print('num of nodes attrs', len(all_attributes['nodes_highway']))
print('num of edges attrs', num_edges_attrs)

num of nodes attrs 21
num of edges attrs 37


#### Seattle

In [3]:
data = pd.read_csv('../data/downstream_tasks/airbnb/seattle/airbnb_seattle_cleaned.csv')
gdf_airbnb_seattle = gpd.GeoDataFrame(
            data.drop(["latitude", "longitude"], axis=1),
            geometry=gpd.points_from_xy(x=data["longitude"], y=data["latitude"]),
            crs="EPSG:4326",
        )

gdf_airbnb_seattle['airbnb'] = 1

In [4]:
G =ox.graph.graph_from_place("Seattle, United States", network_type="drive", simplify=False)
gdf_nodes_seattle, gdf_edges_seattle = ox.graph_to_gdfs(G)

In [7]:
graph_embedder = OSMnxGraph(gdf_airbnb_seattle, gdf_nodes_seattle, gdf_edges_seattle, y_column_name='airbnb')

osmnx_nodes = graph_embedder.get_node_attrs()
display(osmnx_nodes)
print(len(osmnx_nodes.columns.unique()))
print(osmnx_nodes.columns.unique())
osmnx_nodes.to_parquet('../data/organized_graphs/seattle/nodes.parquet', compression=None)

nodes_test = gpd.read_parquet('../data/organized_graphs/seattle/nodes.parquet')


assert_geodataframe_equal(osmnx_nodes, nodes_test, check_like=True)


osmnx_edges = graph_embedder.get_edge_attrs()
display(osmnx_edges)
print(len(osmnx_edges.columns.unique()))
print(osmnx_edges.columns.unique())
osmnx_edges.dtypes
osmnx_edges.to_parquet('../data/organized_graphs/seattle/edges.parquet', compression=None)


edges_test = gpd.read_parquet('../data/organized_graphs/seattle/edges.parquet')

assert_geodataframe_equal(osmnx_edges, edges_test, check_like=True)


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,street_count,geometry,bus_stop,crossing,give_way,milestone,mini_roundabout,motorway_junction,red_light_camera,speed_camera,...,traffic_signals,traffic_signals;crossing,turning_circle,turning_loop,passing_place,street_lamp,stop;crossing,proposed,priority,construction
osmid,,,,,,,,,,,,,,,,,,,,,
29445653,2,POINT (-122.31969 47.64248),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445655,2,POINT (-122.32113 47.64217),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445656,2,POINT (-122.32184 47.64178),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445657,2,POINT (-122.32204 47.64160),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29445659,2,POINT (-122.32219 47.64140),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11932454269,2,POINT (-122.29709 47.57016),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11932454270,2,POINT (-122.29666 47.57139),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11932454288,2,POINT (-122.29712 47.57187),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


23
Index(['street_count', 'geometry', 'bus_stop', 'crossing', 'give_way',
       'milestone', 'mini_roundabout', 'motorway_junction', 'red_light_camera',
       'speed_camera', 'stop', 'stop_line', 'toll_gantry', 'traffic_signals',
       'traffic_signals;crossing', 'turning_circle', 'turning_loop',
       'passing_place', 'street_lamp', 'stop;crossing', 'proposed', 'priority',
       'construction'],
      dtype='object')


/home/grymar/studia/gradient/notebooks/../src/graph/create_osmnx_graph.py:108: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  attrs = attrs.explode(col)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/grymar/studia/gradient/env/lib/pyth

oneway  lanes  maxspeed  reversed  length  \
u           v          key                                                
29445653    4433425415 0   0       1      2        97         0   20.39   
29445655    4433425410 0   0       1      2        97         0   12.41   
29445656    4433425404 0   0       1      2        97         0   14.23   
29445657    1864943659 0   0       1      2        97         0   14.65   
29445659    4433425403 0   0       1      2        97         0   12.50   
...                              ...    ...       ...       ...     ...   
11932454288 53131439   0   0       0      2        36         0    9.70   
            53098734   0   0       0      2        36         1   53.94   
11933647008 9426316116 0   0       0      2        40         0    5.31   
            410778999  0   0       0      2        40         1    4.09   
11936497055 4623146258 0   0       1      2        40         0   38.20   

                              width  \
u           v          key            
29445653    4433425415 0   0  12.26   
29445655    4433425410 0   0  12.26   
29445656    4433425404 0   0  12.26   
29445657    1864943659 0   0  12.26   
29445659    4433425403 0   0  12.26   
...                             ...   
11932454288 53131439   0   0  12.26   
            53098734   0   0  12.26   
11933647008 9426316116 0   0  12.26   
            410778999  0   0  12.26   
11936497055 4623146258 0   0  12.26   

                                                                       geometry  \
u           v          key                                                        
29445653    4433425415 0   0  LINESTRING (-122.31969 47.64248, -122.31943 47...   
29445655    4433425410 0   0  LINESTRING (-122.32113 47.64217, -122.32098 47...   
29445656    4433425404 0   0  LINESTRING (-122.32184 47.64178, -122.32171 47...   
29445657    1864943659 0   0  LINESTRING (-122.32204 47.64160, -122.32193 47...   
29445659    4433425403 0   0  LINESTRING (-122.32219 47.64140, -122.32212 47...   
...                                                                         ...   
11932454288 53131439   0   0  LINESTRING (-122.29712 47.57187, -122.29701 47...   
            53098734   0   0  LINESTRING (-122.29712 47.57187, -122.29778 47...   
11933647008 9426316116 0   0  LINESTRING (-122.33324 47.70133, -122.33330 47...   
            410778999  0   0  LINESTRING (-122.33324 47.70133, -122.33318 47...   
11936497055 4623146258 0   0  LINESTRING (-122.28579 47.54674, -122.28596 47...   

                              highway_busway  highway_living_street  \
u           v          key                                            
29445653    4433425415 0   0               0                      0   
29445655    4433425410 0   0               0                      0   
29445656    4433425404 0   0               0                      0   
29445657    1864943659 0   0               0                      0   
29445659    4433425403 0   0               0                      0   
...                                      ...                    ...   
11932454288 53131439   0   0               0                      0   
            53098734   0   0               0                      0   
11933647008 9426316116 0   0               0                      0   
            410778999  0   0               0                      0   
11936497055 4623146258 0   0               0                      0   

                              highway_motorway  ...  junction_roundabout  \
u           v          key                      ...                        
29445653    4433425415 0   0                 0  ...                    0   
29445655    4433425410 0   0                 0  ...                    0   
29445656    4433425404 0   0                 0  ...                    0   
29445657    1864943659 0   0                 0  ...                    0   
29445659    4433425403 0   0                 0  ...                    0   
...       

44
Index(['oneway', 'lanes', 'maxspeed', 'reversed', 'length', 'width',
       'geometry', 'highway_busway', 'highway_living_street',
       'highway_motorway', 'highway_motorway_link', 'highway_primary',
       'highway_primary_link', 'highway_residential', 'highway_secondary',
       'highway_secondary_link', 'highway_stop_line', 'highway_tertiary',
       'highway_tertiary_link', 'highway_trunk', 'highway_trunk_link',
       'highway_unclassified', 'access_bus', 'access_customers',
       'access_destination', 'access_no', 'access_permissive', 'access_permit',
       'access_yes', 'access_designated', 'access_delivery',
       'access_residents', 'junction_circular', 'junction_intersection',
       'junction_roundabout', 'junction_spui', 'junction_jughandle',
       'bridge_movable', 'bridge_viaduct', 'bridge_yes',
       'bridge_low_water_crossing', 'tunnel_building_passage', 'tunnel_yes',
       'tunnel_no'],
      dtype='object')


#### New York

In [8]:
data = pd.read_csv('../data/downstream_tasks/airbnb/new_york/airbnb_new_york_cleaned.csv')
gdf_airbnb_new_york = gpd.GeoDataFrame(
            data.drop(["latitude", "longitude"], axis=1),
            geometry=gpd.points_from_xy(x=data["longitude"], y=data["latitude"]),
            crs="EPSG:4326",
        )

gdf_airbnb_new_york['airbnb'] = 1

In [9]:
G =ox.graph.graph_from_place("New York City, United States", network_type="drive", simplify=False)
gdf_nodes_ny, gdf_edges_ny = ox.graph_to_gdfs(G)

In [10]:
graph_embedder_ny = OSMnxGraph(gdf_airbnb_new_york, gdf_nodes_ny, gdf_edges_ny, y_column_name='airbnb')

osmnx_nodes_ny = graph_embedder_ny.get_node_attrs()
display(osmnx_nodes_ny)
print(len(osmnx_nodes_ny.columns.unique()))
print(osmnx_nodes_ny.columns.unique())
osmnx_nodes_ny.to_parquet('../data/organized_graphs/new_york/nodes.parquet', compression=None)

nodes_test = gpd.read_parquet('../data/organized_graphs/new_york/nodes.parquet')


assert_geodataframe_equal(osmnx_nodes_ny, nodes_test, check_like=True)


osmnx_edges_ny = graph_embedder.get_edge_attrs()
display(osmnx_edges_ny)
print(len(osmnx_edges_ny.columns.unique()))
print(osmnx_edges_ny.columns.unique())
osmnx_edges_ny.to_parquet('../data/organized_graphs/new_york/edges.parquet', compression=None)
edges_test = gpd.read_parquet('../data/organized_graphs/new_york/edges.parquet')

assert_geodataframe_equal(osmnx_edges_ny, edges_test, check_like=True)

/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,street_count,geometry,bus_stop,crossing,give_way,motorway_junction,priority,speed_camera,stop,stop;crossing,...,traffic_signals;crossing,turning_circle,turning_loop,mini_roundabout,passing_place,red_light_camera,stop_line,proposed,milestone,construction
osmid,,,,,,,,,,,,,,,,,,,,,
33751990,2,POINT (-73.82416 40.78696),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33751991,2,POINT (-73.82526 40.78187),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33751992,2,POINT (-73.82708 40.77840),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33751993,2,POINT (-73.82931 40.77479),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33751994,2,POINT (-73.83150 40.77266),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11937461981,2,POINT (-73.89788 40.73588),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11937461982,2,POINT (-73.89773 40.73593),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11937720191,2,POINT (-73.95376 40.82528),0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


23
Index(['street_count', 'geometry', 'bus_stop', 'crossing', 'give_way',
       'motorway_junction', 'priority', 'speed_camera', 'stop',
       'stop;crossing', 'street_lamp', 'toll_gantry', 'traffic_signals',
       'traffic_signals;crossing', 'turning_circle', 'turning_loop',
       'mini_roundabout', 'passing_place', 'red_light_camera', 'stop_line',
       'proposed', 'milestone', 'construction'],
      dtype='object')


/home/grymar/studia/gradient/notebooks/../src/graph/create_osmnx_graph.py:108: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  attrs = attrs.explode(col)


oneway  lanes  maxspeed  reversed  length  \
u           v          key                                                  
29445653    4433425415 0   0 0       1      2        97         0   20.39   
29445655    4433425410 0   0 0       1      2        97         0   12.41   
29445656    4433425404 0   0 0       1      2        97         0   14.23   
29445657    1864943659 0   0 0       1      2        97         0   14.65   
29445659    4433425403 0   0 0       1      2        97         0   12.50   
...                                ...    ...       ...       ...     ...   
11932454288 53131439   0   0 0       0      2        36         0    9.70   
            53098734   0   0 0       0      2        36         1   53.94   
11933647008 9426316116 0   0 0       0      2        40         0    5.31   
            410778999  0   0 0       0      2        40         1    4.09   
11936497055 4623146258 0   0 0       1      2        40         0   38.20   

                                width  highway_busway  highway_living_street  \
u           v          key                                                     
29445653    4433425415 0   0 0  12.26               0                      0   
29445655    4433425410 0   0 0  12.26               0                      0   
29445656    4433425404 0   0 0  12.26               0                      0   
29445657    1864943659 0   0 0  12.26               0                      0   
29445659    4433425403 0   0 0  12.26               0                      0   
...                               ...             ...                    ...   
11932454288 53131439   0   0 0  12.26               0                      0   
            53098734   0   0 0  12.26               0                      0   
11933647008 9426316116 0   0 0  12.26               0                      0   
            410778999  0   0 0  12.26               0                      0   
11936497055 4623146258 0   0 0  12.26               0                      0   

                                highway_motorway  highway_motorway_link  ...  \
u           v          key                                               ...   
29445653    4433425415 0   0 0                 0                      1  ...   
29445655    4433425410 0   0 0                 0                      1  ...   
29445656    4433425404 0   0 0                 0                      1  ...   
29445657    1864943659 0   0 0                 0                      1  ...   
29445659    4433425403 0   0 0                 0                      1  ...   
...                                          ...                    ...  ...   
11932454288 53131439   0   0 0                 0                      0  ...   
            53098734   0   0 0                 0                      0  ...   
11933647008 9426316116 0   0 0                 0                      0  ...   
            410778999  0   0 0                 0                      0  ...   
11936497055 4623146258 0   0 0                 0                      0  ...   

                                junction_spui  junction_jughandle  \
u           v          key                                          
29445653    4433425415 0   0 0              0                   0   
29445655    4433425410 0   0 0              0                   0   
29445656    4433425404 0   0 0              0                   0   
29445657    1864943659 0   0 0              0                   0   
29445659    4433425403 0   0 0              0                   0   
...                                       ...                 ...   
11932454288 53131439   0   0 0              0                   0   
            53098734   0   0 0              0                   0   
11933647008 9426316116 0   0 0              0                   0   
            410778999  0   0 0              0                   0   
11936497055 4623146258 0   0 0              0                   0   

                                bridge_movable  br

44
Index(['oneway', 'lanes', 'maxspeed', 'reversed', 'length', 'width',
       'highway_busway', 'highway_living_street', 'highway_motorway',
       'highway_motorway_link', 'highway_primary', 'highway_primary_link',
       'highway_residential', 'highway_secondary', 'highway_secondary_link',
       'highway_stop_line', 'highway_tertiary', 'highway_tertiary_link',
       'highway_trunk', 'highway_trunk_link', 'highway_unclassified',
       'access_bus', 'access_customers', 'access_destination', 'access_no',
       'access_permissive', 'access_permit', 'access_yes', 'access_designated',
       'access_delivery', 'access_residents', 'junction_circular',
       'junction_intersection', 'junction_roundabout', 'junction_spui',
       'junction_jughandle', 'bridge_movable', 'bridge_viaduct', 'bridge_yes',
       'bridge_low_water_crossing', 'tunnel_building_passage', 'tunnel_yes',
       'tunnel_no', 'geometry'],
      dtype='object')


## PL

In [17]:

for city in ['warszawa', 'krakow', 'wroclaw', 'poznan', 'szczecin']:
    df_accidents = pd.read_csv("../data/wypadki-pl/accidents.csv")
    df_accidents.drop(df_accidents[(df_accidents['mie_nazwa'] != city)].index, inplace=True)
    df_accidents.drop(columns='uczestnicy', inplace=True)
    geometry = [Point(xy) for xy in zip(df_accidents['wsp_gps_x'], df_accidents['wsp_gps_y'])]
    gdf_accidents = gpd.GeoDataFrame(df_accidents, geometry=geometry)
    gdf_accidents.drop(columns=['wsp_gps_x', 'wsp_gps_y'], inplace=True)

    cs = '[out:json][timeout:180][date:"2022-01-01T00:00:00Z"]'
    ox.settings.overpass_settings = cs
    G =ox.graph.graph_from_place(f"{city}, Polska", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)


    graph_embedder = OSMnxGraph(gdf_accidents, gdf_nodes, gdf_edges, y_column_name='mie_nazwa')

    osmnx_nodes = graph_embedder.get_node_attrs()
    display(osmnx_nodes)
    print(len(osmnx_nodes.columns.unique()))
    print(osmnx_nodes.columns.unique())
    osmnx_nodes.to_parquet(f'../data/organized_graphs/{city}/nodes.parquet', compression=None)

    nodes_test = gpd.read_parquet(f'../data/organized_graphs/{city}/nodes.parquet')


    assert_geodataframe_equal(osmnx_nodes, nodes_test, check_like=True)


    osmnx_edges = graph_embedder.get_edge_attrs()
    display(osmnx_edges)
    print(len(osmnx_edges.columns.unique()))
    print(osmnx_edges.columns.unique())
    osmnx_edges.to_parquet(f'../data/organized_graphs/{city}/edges.parquet', compression=None)
    edges_test = gpd.read_parquet(f'../data/organized_graphs/{city}/edges.parquet')

    assert_geodataframe_equal(osmnx_edges, edges_test, check_like=True)


/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,street_count,geometry,bus_stop,construction,crossing,give_way,milestone,mini_roundabout,motorway_junction,passing_place,...,traffic_signals,traffic_signals;crossing,turning_circle,turning_loop,red_light_camera,street_lamp,stop;crossing,stop_line,priority,toll_gantry
osmid,,,,,,,,,,,,,,,,,,,,,
26063726,5,POINT (21.01745 52.15279),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26063729,2,POINT (21.01767 52.14244),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26063848,4,POINT (20.99139 52.15757),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26063849,2,POINT (20.99202 52.15808),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26063850,2,POINT (20.99219 52.15847),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9382705507,2,POINT (21.22122 52.18005),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9382705510,2,POINT (21.22490 52.18195),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9382705512,2,POINT (21.22329 52.18057),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


23
Index(['street_count', 'geometry', 'bus_stop', 'construction', 'crossing',
       'give_way', 'milestone', 'mini_roundabout', 'motorway_junction',
       'passing_place', 'proposed', 'speed_camera', 'stop', 'traffic_signals',
       'traffic_signals;crossing', 'turning_circle', 'turning_loop',
       'red_light_camera', 'street_lamp', 'stop;crossing', 'stop_line',
       'priority', 'toll_gantry'],
      dtype='object')


/home/grymar/studia/gradient/notebooks/../src/graph/create_osmnx_graph.py:108: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  attrs = attrs.explode(col)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/grymar/studia/gradient/env/lib/pyth

oneway  lanes  reversed  length  maxspeed  width  \
u          v          key                                                       
26063726   31717945   0   0       1      4         0   25.02        46   5.14   
           270313043  0   0       1      4         0   26.03        60   5.14   
26063729   2382490050 0   0       1      3         0   38.50        60   5.14   
26063848   8043122320 0   0       0      2         1    7.27        46   5.14   
           1268494441 0   0       0      2         0   11.01        46   5.14   
...                             ...    ...       ...     ...       ...    ...   
9382705512 9382705513 0   0       0      2         1   17.94        46   5.14   
9382705513 9382705512 0   0       0      2         0   17.94        46   5.14   
           3797896895 0   0       0      2         1   33.00        46   5.14   
9382705514 309168721  0   0       0      2         0   15.61        46   5.14   
           8167152391 0   0       0      2         1   13.42        46   5.14   

                                                                      geometry  \
u          v          key                                                        
26063726   31717945   0   0  LINESTRING (21.01745 52.15279, 21.01780 52.15271)   
           270313043  0   0  LINESTRING (21.01745 52.15279, 21.01745 52.15255)   
26063729   2382490050 0   0  LINESTRING (21.01767 52.14244, 21.01767 52.14210)   
26063848   8043122320 0   0  LINESTRING (20.99139 52.15757, 20.99129 52.15759)   
           1268494441 0   0  LINESTRING (20.99139 52.15757, 20.99127 52.15751)   
...                                                                        ...   
9382705512 9382705513 0   0  LINESTRING (21.22329 52.18057, 21.22343 52.18071)   
9382705513 9382705512 0   0  LINESTRING (21.22343 52.18071, 21.22329 52.18057)   
           3797896895 0   0  LINESTRING (21.22343 52.18071, 21.22370 52.18096)   
9382705514 309168721  0   0  LINESTRING (21.22537 52.18231, 21.22559 52.18230)   
           8167152391 0   0  LINESTRING (21.22537 52.18231, 21.22517 52.18232)   

                             highway_living_street  highway_primary  \
u          v          key                                             
26063726   31717945   0   0                      0                0   
           270313043  0   0                      0                1   
26063729   2382490050 0   0                      0                1   
26063848   8043122320 0   0                      0                0   
           1268494441 0   0                      0                0   
...                                            ...              ...   
9382705512 9382705513 0   0                      0                0   
9382705513 9382705512 0   0                      0                0   
           3797896895 0   0                      0                0   
9382705514 309168721  0   0                      0                0   
           8167152391 0   0                      0                0   

                             highway_primary_link  ...  junction_spui  \
u          v          key                          ...                  
26063726   31717945   0   0                     0  ...              0   
           270313043  0   0                     0  ...              0   
26063729   2382490050 0   0                     0  ...              0   
26063848   8043122320 0   0                     0  ...              0   
           1268494441 0   0                     0  ...              0   
...                                           ...  ...            ...   
9382705512 9382705513 0   0                     0  ...              0   
9382705513 9382705512 0   0                     0  ...              0   
           3797896895 0   0                     0  ...              0   
9382705514 309168721  0   0                     0  ...              0   
           8167152391 0   0                     0  ...              0   

                             junct

...

,street_count,geometry,bus_stop,construction,crossing,give_way,mini_roundabout,motorway_junction,passing_place,stop,...,red_light_camera,street_lamp,stop;crossing,stop_line,proposed,priority,turning_loop,traffic_signals;crossing,milestone,speed_camera
osmid,,,,,,,,,,,,,,,,,,,,,
253316750,3,POINT (14.73497 53.43076),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253804823,2,POINT (14.65004 53.37467),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253804824,4,POINT (14.64961 53.37325),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253804825,2,POINT (14.64949 53.37306),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253804826,2,POINT (14.64875 53.37296),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9376390593,2,POINT (14.70067 53.37214),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9379226721,2,POINT (14.48775 53.41006),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9381932426,2,POINT (14.53368 53.42308),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


23
Index(['street_count', 'geometry', 'bus_stop', 'construction', 'crossing',
       'give_way', 'mini_roundabout', 'motorway_junction', 'passing_place',
       'stop', 'traffic_signals', 'turning_circle', 'toll_gantry',
       'red_light_camera', 'street_lamp', 'stop;crossing', 'stop_line',
       'proposed', 'priority', 'turning_loop', 'traffic_signals;crossing',
       'milestone', 'speed_camera'],
      dtype='object')


/home/grymar/studia/gradient/notebooks/../src/graph/create_osmnx_graph.py:108: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  attrs = attrs.explode(col)
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/grymar/studia/gradient/env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/grymar/studia/gradient/env/lib/pyth

oneway  reversed  length  lanes  maxspeed  width  \
u          v          key                                                       
253316750  2078087067 0   0       0         1   76.22      2        46   7.54   
           7835989839 0   0       0         0  193.06      2        46   7.54   
           4621471040 0   0       1         0    6.21      2        46   7.54   
253804823  2158232327 0   0       0         0   30.60      2        46   7.54   
           301130111  0   0       0         1   42.81      2        46   7.54   
...                             ...       ...     ...    ...       ...    ...   
9381932426 991597549  0   0       0         1    5.70      2        50   7.54   
9381932463 1811107779 0   0       0         0    2.72      2        50   7.54   
           7006989335 0   0       0         1   91.38      2        50   7.54   
9381932467 1744171558 0   0       0         0    1.97      2        50   7.54   
           7006989332 0   0       0         1   27.12      2        50   7.54   

                                                                      geometry  \
u          v          key                                                        
253316750  2078087067 0   0  LINESTRING (14.73497 53.43076, 14.73552 53.43015)   
           7835989839 0   0  LINESTRING (14.73497 53.43076, 14.73356 53.43227)   
           4621471040 0   0  LINESTRING (14.73497 53.43076, 14.73505 53.43078)   
253804823  2158232327 0   0  LINESTRING (14.65004 53.37467, 14.64995 53.37440)   
           301130111  0   0  LINESTRING (14.65004 53.37467, 14.65015 53.37505)   
...                                                                        ...   
9381932426 991597549  0   0  LINESTRING (14.53368 53.42308, 14.53369 53.42313)   
9381932463 1811107779 0   0  LINESTRING (14.53914 53.42454, 14.53910 53.42454)   
           7006989335 0   0  LINESTRING (14.53914 53.42454, 14.54047 53.42433)   
9381932467 1744171558 0   0  LINESTRING (14.53838 53.42324, 14.53837 53.42323)   
           7006989332 0   0  LINESTRING (14.53838 53.42324, 14.53848 53.42348)   

                             highway_living_street  highway_motorway  \
u          v          key                                              
253316750  2078087067 0   0                      0                 0   
           7835989839 0   0                      0                 0   
           4621471040 0   0                      0                 0   
253804823  2158232327 0   0                      0                 0   
           301130111  0   0                      0                 0   
...                                            ...               ...   
9381932426 991597549  0   0                      0                 0   
9381932463 1811107779 0   0                      0                 0   
           7006989335 0   0                      0                 0   
9381932467 1744171558 0   0                      0                 0   
           7006989332 0   0                      0                 0   

                             highway_motorway_link  ...  \
u          v          key                           ...   
253316750  2078087067 0   0                      0  ...   
           7835989839 0   0                      0  ...   
           4621471040 0   0                      0  ...   
253804823  2158232327 0   0                      0  ...   
           301130111  0   0                      0  ...   
...                                            ...  ...   
9381932426 991597549  0   0                      0  ...   
9381932463 1811107779 0   0                      0  ...   
           7006989335 0   0                      0  ...   
9381932467 1744171558 0   0                      0  ...   
           7006989332 0   0                      0  ...   

                             junction_intersection  junction_circular  \
u          v          key                                               
253316750  2078087067 0   0                      0       

45
Index(['oneway', 'reversed', 'length', 'lanes', 'maxspeed', 'width',
       'geometry', 'highway_living_street', 'highway_motorway',
       'highway_motorway_link', 'highway_primary', 'highway_primary_link',
       'highway_residential', 'highway_secondary', 'highway_secondary_link',
       'highway_tertiary', 'highway_trunk', 'highway_unclassified',
       'highway_trunk_link', 'highway_busway', 'highway_tertiary_link',
       'highway_stop_line', 'access_designated', 'access_destination',
       'access_no', 'access_permissive', 'access_permit', 'access_yes',
       'access_customers', 'access_bus', 'access_delivery', 'access_residents',
       'junction_roundabout', 'junction_yes', 'junction_spui',
       'junction_intersection', 'junction_circular', 'junction_jughandle',
       'bridge_viaduct', 'bridge_yes', 'bridge_low_water_crossing',
       'bridge_movable', 'tunnel_building_passage', 'tunnel_yes', 'tunnel_no'],
      dtype='object')
